In [4]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import os
from tensorflow.keras.applications import EfficientNetV2M
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

In [2]:
DROPOUT_RATE=0.2
BATCH_SIZE = 32
EPOCHS=10
NUM_CLASSES = 2

In [5]:

base_model = EfficientNetV2M(
    include_top=False, 
    weights='imagenet',
    input_shape = (250,250,3)
    )

output_of_pretrained = base_model.output
x = Flatten()(output_of_pretrained)
x = Dense(2048, activation='relu', name='fc01')(x)
x = Dropout(DROPOUT_RATE)(x)
x = Dense(1024, activation='relu', name='fc02')(x)
x = Dropout(DROPOUT_RATE)(x)

predictions = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs = base_model.input, outputs = predictions, name='myModel')
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

train_datagenerator = ImageDataGenerator(
    rescale = 1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
validation_generator = ImageDataGenerator(
    rescale=1./255
)

train_data = train_datagenerator.flow_from_directory(
    directory = './divided_smoker_dataset/Training',
    target_size = (250,250),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = BATCH_SIZE, 
    shuffle = True,
    seed=42
)
test_data = validation_generator.flow_from_directory(
    directory = './divided_smoker_dataset/Validation',
    target_size = (250,250),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = BATCH_SIZE, 
    shuffle = False
)
class_labels = list(train_data.class_indices.keys())
print("Class Labels:", class_labels)
model.fit(train_data, epochs=EPOCHS, validation_data=test_data)

model.save('fine_tuned_efficientnet.h5')

214201816/214201816 [==============================] - 14s 0us/step
Found 716 images belonging to 2 classes.
Found 180 images belonging to 2 classes.
Class Labels: ['nonsmoke', 'smoke']
Epoch 1/10
23/23 [==============================] - 595s 23s/step - loss: 8.5776 - accuracy: 0.5265 - val_loss: 1.5162 - val_accuracy: 0.5000
Epoch 2/10
23/23 [==============================] - 553s 24s/step - loss: 1.0213 - accuracy: 0.5908 - val_loss: 0.7496 - val_accuracy: 0.5667
Epoch 3/10
23/23 [==============================] - 511s 22s/step - loss: 0.5592 - accuracy: 0.7500 - val_loss: 0.4352 - val_accuracy: 0.8056
Epoch 4/10
23/23 [==============================] - 499s 22s/step - loss: 0.4394 - accuracy: 0.8031 - val_loss: 0.4637 - val_accuracy: 0.7944
Epoch 5/10
23/23 [==============================] - 494s 22s/step - loss: 0.4018 - accuracy: 0.8226 - val_loss: 0.3661 - val_accuracy: 0.8389
Epoch 6/10
23/23 [==============================] - 490s 21s/step - loss: 0.2706 - accuracy: 0.8883 - va

C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
